## Functions and Chat Completion
- [Create our own Code Interpreter | A hacker's guide to Language Models ](https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb)
- [GPT - OpenAI API ](https://platform.openai.com/docs/guides/gpt/function-calling)
- [Function-calling with an OpenAPI specification | OpenAI Cookbook](https://cookbook.openai.com/examples/function_calling_with_an_openapi_spec)

Jeremy Howard provided an overview of using ChatGPT to create you own "Code Interpreter" that can be used by LLMs to call python code.

See:
https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb

In [ ]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [ ]:
from fastcore.utils import nested_idx
def response(compl): print(nested_idx(compl, 'choices', 0, 'message', 'content'))

In [ ]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [ ]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [ ]:
schema(sums)

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_47171/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

We don't want to disclose our OPEN_API_KEY to the world, so a good best practice is to put the key in a .env file that is ignored in the .gitignore so we don't accidentaly import it into GitHub. We use the python-dotenv package to import the OPENAI_API_KEY variable from the environment and assign it to the openai.api_key. For codespaces and docker containers, there are methods to handle secrets that get set to environment variables at startup so as to not accidentally leak information.

In [ ]:
import openai
from openai import ChatCompletion,Completion
from dotenv import load_dotenv, find_dotenv
from fastcore.utils import *
from pathlib import Path
from os import getenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

if not load_dotenv(".env"):
    raise ValueError("EnvironmentError: Failed to load `.env`")

api_key = getenv("OPENAI_API_KEY") or ""

if not api_key:
    raise ValueError("EnvironmentError: Failed to load `OPENAI_API_KEY`")

openai.api_key = api_key



In [ ]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [ ]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_47171/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [ ]:
m = c.choices[0].message
m

<OpenAIObject> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "sums",
    "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
  }
}

In [ ]:
k = m.function_call.arguments
print(k)

{
  "a": 6,
  "b": 3
}


In [ ]:
funcs_ok = {'sums', 'python'}

In [ ]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [ ]:
call_func(c)

9

In [ ]:
import ast

In [ ]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [ ]:
run("""
a=1
b=2
a+b
""")

3

In [ ]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [ ]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_22254/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [ ]:
call_func(c)

KeyboardInterrupt: Interrupted by user

In [ ]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    functions=[schema(python)],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "python", "content": "479001600"}])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_42954/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [ ]:
response(c)

12 factorial, denoted as 12!, is equal to 479,001,600.


In [ ]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_42954/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [ ]:
response(c)


The capital of France is Paris.


## LLama based agents and Llama.cpp functions
[Zeng, Aohan, Mingdao Liu, Rui Lu, Bowen Wang, Xiao Liu, Yuxiao Dong, and Jie Tang. 2023. “AgentTuning: Enabling Generalized Agent Abilities for LLMs.” arXiv. http://arxiv.org/abs/2310.12823.
](http://arxiv.org/abs/2310.12823)

- [The Bloke Quantized](https://huggingface.co/TheBloke/agentlm-7B-GGUF/blob/main/agentlm-7b.Q4_K_M.gguf)

- [lamma.cpp Grammers](https://github.com/ggerganov/llama.cpp/tree/master/grammars)
- [Introducing Code Llama, a state-of-the-art large language model for coding](https://ai.meta.com/blog/code-llama-large-language-model-coding/)
- [lamma.cpp Python](https://github.com/abetlen/llama-cpp-python)
- [Implementation of Functions using Lamma.cpp grammar](https://github.com/teleprint-me/py.gpt.prompt/blob/main/docs/notebooks/llama_cpp_grammar_api.ipynb)

In [ ]:
import json
import os
import pathlib
import requests
from pprint import pprint
from llama_cpp import ChatCompletionMessage, Llama, LlamaGrammar

In [ ]:
llm = Llama(model_path="./models/agentlm-70b.Q5_K_S.gguf",n_gpu_layers=1,n_gqa=8)

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from ./models/agentlm-70b.Q5_K_S.gguf (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32256,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q5_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192,  8192,     1,     1 ]
lla

In [ ]:
llm = Llama(model_path="./models/agentlm-70b.Q5_K_S.gguf")

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from ./models/agentlm-70b.Q5_K_S.gguf (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32256,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q5_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192,  8192,     1,     1 ]
lla

In [ ]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=100, stop=["Q:", "\n"], echo=True)
print(output)

{'id': 'cmpl-d0ce17c2-bd0c-4cbd-83db-d6efafd29098', 'object': 'text_completion', 'created': 1698427338, 'model': './models/agentlm-70b.Q5_K_S.gguf', 'choices': [{'text': 'Q: Name the planets in the solar system? A: 1. Mercury 2. Venus 3. Earth 4. Mars 5. Jupiter', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 15, 'completion_tokens': 23, 'total_tokens': 38}}



llama_print_timings:        load time =  7858.44 ms
llama_print_timings:      sample time =    16.58 ms /    23 runs   (    0.72 ms per token,  1387.21 tokens per second)
llama_print_timings: prompt eval time =  7858.00 ms /    15 tokens (  523.87 ms per token,     1.91 tokens per second)
llama_print_timings:        eval time = 11177.68 ms /    22 runs   (  508.08 ms per token,     1.97 tokens per second)
llama_print_timings:       total time = 19083.64 ms


In [ ]:
llama_grammar = LlamaGrammar.from_file("json.gbnf")
print(llama_grammar)

root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number_28 
number_27 ::= [-

from_string grammar:

